In [ ]:
import re
import numpy as np
from scripts.scrape import get_data, get_test_data



# Day 8

In [ ]:
data = get_data(8)
test = get_test_data(8)
data

In [ ]:
def run_data(data):
    visited = np.zeros(len(data))
    accumulator = 0
    idx = 0
    while(visited[idx] == 0):
        visited[idx] = 1
        instruction, value = re.split(' ', data[idx])
        value = int(value)
        if instruction == 'acc':
            accumulator += value
            idx += 1
        elif instruction == 'jmp':
            idx += value
        else:
            idx +=1
        if idx == len(data) - 1:
            print('finished')
            print(accumulator)
            return(None)
    return(accumulator)
    

In [ ]:
run_data(data)

In [ ]:
for idx in range(len(data)):
    instruction, value = re.match('([a-z]{3}) \+?(-?[0-9]+)', data[idx]).groups()
    new_data = data.copy()
    if instruction == 'jmp':
        new_data[idx] = 'nop {}'.format(value)
    elif instruction == 'nop':
        new_data[idx] = 'jmp {}'.format(value)
    elif instruction == 'acc':
        next
    cont = run_data(new_data)
    if(not cont):
        break

# Day 9

In [ ]:
data = get_data(9)
test = get_test_data(9)
data

In [ ]:
for i in range(25, 1000):
    matrix = nums[(i-25):i].reshape(1, 25) + nums[(i-25):i].reshape(25, 1)
    if(nums[i] not in matrix):
        print(nums[i])
        break
        
    

In [ ]:
def break_code(n, nums):
    for i in range(len(nums)):
        min_idx = i
        minimum = nums[i]
        s = 0
        while(s < n):
            s+= nums[i]
            i += 1
            if s == n:
                return(min(nums[min_idx:i]), max(nums[min_idx:i]))


In [ ]:
sum(break_code(675280050, nums))

# Day 10

In [ ]:
data = get_data(10)
test = get_test_data(10)
data

In [ ]:
data = [int(d) for d in data]
data.append(max(data) + 3)
data.append(0)
data.sort()
print(sum(np.diff(data) == 1), (sum(np.diff(data) == 3)))

In [ ]:
sum(np.diff(data) == 1) * (sum(np.diff(data) == 3))

### Part 2

In [ ]:
transitions = np.diff(data)

In [ ]:
runs = []
i = 0
run = 1
while(i < len(transitions)):

    if transitions[i] == 1:
        run +=1
    else:
        runs.append(run)
        run = 1
    i +=1


In [ ]:
min(runs), max(runs)

### There is a max runs of 5. 

A run is a set of 1-jump transitions in a row. The number of permutations for a row is a Tribonacci sequence. WHICH I DID NOT KNOW UNTIL AFTER I FINISHED. 

1:
- 1

2:
- 1 2

3:
- 1 2 3
- 1 3

4: 
- 1 2 3 4
- 1 3 4
- 1 2 4
- 1 4

5:
- 1 2 3 4 5
- 1 3 4 5
- 1 2 4 5
- 1 4 5
- 1 2 3 5
- 1 3 5
- 1 2 5


In [ ]:
mapping = {1:1,
          2:1,
          3:2,
          4:4,
          5:7}


In [ ]:
np.prod([mapping[r] for r in runs])

# Day 11

This one was so frustrating for me. I recognized as a version of Conway's Game of Life, but the example says: 

    At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state!
    
And I interpreted this to mean "after 5 applications, the system stabilizes and rules don't apply". So I ran mine for 5 iterations instead of "to stability" and I couldn't figure out why it wasn't giving me the right answer. Ended up reading the question over and over like 60 times before I realized I had misinterpreted it. I'm not sure how I improve such that I understand the question correctly, especially since my code worked on the test case.

In [ ]:
data = get_data(11)
test = get_test_data(11)
data

In [ ]:
def get_instruction(x, y, data):
    padded_data = np.pad(data, mode='constant', pad_width=1)    
    sub = padded_data[x:x+3, y:y+3]
    sub[1,1] = 0
    
    this_seat = data[x,y]
    occupied_neighbors = np.sum(sub == '#')
    
    # Fill if empty:
    if (occupied_neighbors == 0) & (this_seat == 'L'):
        return 'sit'
    if (occupied_neighbors >= 4) & (this_seat == '#'):
        return 'leave'
    else: 
        return -1

In [ ]:
data = np.array([list(d) for d in get_data(11)])
starting_empties = np.where(data=='.')
instruction = np.empty([data.shape[0], data.shape[1]]).astype(str)

print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
last = data.copy()

while(True):
    for i in range(0, data.shape[0]):
        for j in range(0,  data.shape[1]):
            instruction[i, j] = get_instruction(i, j,  data)
    data[instruction == 'sit'] = '#'
    data[instruction == 'leave'] = 'L'
    print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
    if (last == data).all():
        break
    last = data.copy()

In [ ]:
def get_seat(x, y, dir_x, dir_y, data):
    i = 1
    while(True):
        target_x = x + dir_x * i
        target_y = y + dir_y * i
        if (target_x < 0) | (target_x >= data.shape[0]) | (target_y < 0) | (target_y >= data.shape[1]):
            return '.'
        elif (data[target_x, target_y] != '.'):
            return data[target_x, target_y]
        else:
            i+=1
        
def get_instruction_2(x, y, data):

    sub = np.empty([3,3]).astype('str')
    
    for i in np.arange(-1,2):
        for j in np.arange(-1,2):
            if not ((i==0) & (j==0)):
                sub[i+1,j+1] = get_seat(x,y,i,j,data)
    
    this_seat = data[x,y]
    occupied_neighbors = np.sum(sub == '#')
    
    # Fill if empty:
    if (occupied_neighbors == 0) & (this_seat == 'L'):
        return 'sit'
    if (occupied_neighbors >= 5) & (this_seat == '#'):
        return 'leave'
    else: 
        return -1

In [ ]:
data = np.array([list(d) for d in get_data(11)])
starting_empties = np.where(data=='.')
instruction = np.empty([data.shape[0], data.shape[1]]).astype(str)

print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
last = data.copy()

while(True):
    for i in range(0, data.shape[0]):
        for j in range(0,  data.shape[1]):
            instruction[i, j] = get_instruction_2(i, j,  data)
    data[instruction == 'sit'] = '#'
    data[instruction == 'leave'] = 'L'
    print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
    if (last == data).all():
        break
    last = data.copy()

# Day 12

In [ ]:
data = get_data(12)
test = get_test_data(12)
data

In [ ]:
def process(data, bearing):
    instruction, value = re.findall('([A-Z])([0-9]+)', data)[0]
    value = int(value)
    if instruction == 'W':
        return([-value, 0, 0])
    elif instruction == 'E':
        return([value, 0, 0])
    elif instruction == 'N':
        return([0, value, 0])
    elif instruction == 'S':
        return([0, -value, 0])
    elif instruction == 'F':
        return([value * np.cos(bearing * np.pi/180), value *np.sin(bearing * np.pi/180), 0])
    elif instruction == 'L':    
        return([0, 0, value])
    elif instruction == 'R':
        return([0, 0, -value])  



In [ ]:
loc = np.array([0,0,0]).astype('float64')
for d in data:
    loc += np.array(process(d, loc[2]))
    


In [ ]:
np.abs(loc[0]) + np.abs(loc[1])

In [ ]:
def process_2(data, waypoint, coords):
    instruction, value = re.findall('([A-Z])([0-9]+)', data)[0]
    value = int(value)
    radians = value * np.pi/180    
    if instruction == 'W':
        return(waypoint + [-value, 0], coords)
    elif instruction == 'E':
        return(waypoint + [value, 0], coords)
    elif instruction == 'N':
        return(waypoint + [0, value], coords)
    elif instruction == 'S':
        return(waypoint + [0, -value], coords)
    elif instruction == 'F':
        return(waypoint, coords + value * waypoint)
    elif instruction == 'L': 
        transformed = (np.array(((np.cos(radians), -np.sin(radians)), (np.sin(radians), np.cos(radians)))) @ waypoint)
        return(transformed, coords)
    elif instruction == 'R':
        transformed = (np.array(((np.cos(-radians), -np.sin(-radians)), (np.sin(-radians), np.cos(-radians)))) @ waypoint)
        return(transformed, coords)



In [ ]:
waypoint = np.array([10,1]).astype('float64')
coords = np.array([0,0]).astype('float64')
for d in data:
    print(waypoint, coords)
    waypoint, coords = np.array(process_2(d, waypoint, coords))


In [ ]:
np.sum(np.abs(coords))


# Lucky Day 13

Need to go back and learn how to solve the equation myself instead of relying on the package

In [ ]:
data = get_data(13)
test = get_test_data(13)
data

In [ ]:
timestamp = int(data[0])

In [ ]:
buses = data[1]
buses = [int(b) for b in re.split('[,]', buses) if b != 'x']
buses

In [ ]:
arrival_times = [(1 + timestamp // b) * b for b in buses]
best_bus = np.argmin(arrival_times)

In [ ]:
(arrival_times[best_bus] - timestamp) * buses[best_bus]

###  Part2

In [ ]:
constraints = data[1].split(',')

In [ ]:
constraints_mapping = dict(zip([idx for idx, item in enumerate(constraints) if item != 'x'], \
                              [int(item) for idx, item in enumerate(constraints) if item != 'x']))

In [ ]:
constraints_mapping

x[0] * value + (value-key) = t


In [ ]:
from sympy import Matrix
from diophantine import solve

In [ ]:
mat = np.diag(list(constraints_mapping.values()))
mat_new = np.zeros((mat.shape[0], mat.shape[1]+1))
mat_new[:mat.shape[0], :mat.shape[0]] = mat
mat_new[:,-1:] = -1

dioph = Matrix(mat_new)

# targets= Matrix([(-key) % value for key,value in constraints_mapping.items()])
tar = [(key) for key,value in constraints_mapping.items()]
targets= Matrix(tar)

In [ ]:
dioph

In [ ]:
targets

In [ ]:
list(solve(dioph, targets)[0])

The last one is the solution for the timestamp. This is negative, which is useful. We can generate infinitely many solutions by adding a multiple of the product of the other coefficients (bus numbers) divided by their gcd (1, because they are all prime)

In [ ]:
np.prod(list(constraints_mapping.values())) - 346706995564810

In [ ]:
ts = 402251700208309
(ts + np.array(list(constraints_mapping.keys()))) % np.array(list(constraints_mapping.values()))

In [ ]:
# After sleeping. This is way better.
constraints_mapping = [[idx, int(item)] for idx, item in enumerate(constraints) if item != 'x']

def integer_solutions(bus_1, bus_2):
    offset = bus_1[0]
    increment = bus_1[1]
    delay = bus_2[0]
    mod_value = bus_2[1]
    while((offset + delay) % mod_value != 0):
        offset += increment
    return(offset, increment * mod_value)
    
reduce(integer_solutions, constraints_mapping)
    

# Day 14

In [ ]:
data = get_data(14)
test = get_test_data(14)
data

In [ ]:
memory = {}
for item in data:
    instruction, value = re.split(' = ', item)
    if instruction == 'mask':
        mask_1 = np.where(np.array(list(value)) == '1')
        mask_0 = np.where(np.array(list(value)) == '0')
    else:
        idx = re.findall('([0-9]+)', instruction)[0]
        bin_number = bin(int(value))[2:]
        bin_number = (36 - len(bin_number))*'0' + bin_number
        bin_number = np.array(list(bin_number)).astype(int)
        bin_number[mask_1] = 1
        bin_number[mask_0] = 0
        memory[idx] = int(''.join(bin_number.astype(str)), 2)
        
        
        


In [ ]:
np.sum(list(memory.values()))

### Part 2

In [ ]:
test_mask = re.split(' = ', data[0])[1]

In [ ]:
from itertools import combinations

In [ ]:
memory = {}
for item in data:
    instruction, value = re.split(' = ', item)
    if instruction == 'mask':
        mask_1 = np.where(np.array(list(value)) == '1')[0]
        mask_X = np.where(np.array(list(value)) == 'X')[0]        
    else:
        idx = re.findall('([0-9]+)', instruction)[0]
        bin_number = bin(int(idx))[2:]
        bin_number = (36 - len(bin_number))*'0' + bin_number
        bin_number = np.array(list(bin_number)).astype(int)
        
        bin_number[mask_1] = 1
        
        for r in range(len(mask_X)+1):
            for c in combinations(mask_X, r):
                ones = list(c)
                zeros = list(set(mask_X) - set(list(c)))
                bin_number[ones] = 1
                bin_number[zeros] = 0
                memory[int(''.join(bin_number.astype(str)), 2)] = int(value)

In [ ]:
np.sum(list(memory.values()))

### Part 1 Redux

Forgot about bitwise operations


In [ ]:
memory_2 = {}
for item in data:
    instruction, value = re.split(' = ', item)
    if instruction == 'mask':
        mask_1 = int(re.sub('X', '0', value), 2)
        mask_0 = int(re.sub('X', '1', value), 2)
    else:
        idx = re.findall('([0-9]+)', instruction)[0]
        memory_2[idx] = (int(value) | mask_1) & mask_0
        
print(np.sum(list(memory.values())))
print(np.sum(list(memory_2.values()))) 
        
